In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/areeg-library/ArEEG-an-Open-Access-Arabic-Inner-Speech-EEG-Dataset-main/LICENSE
/kaggle/input/areeg-library/ArEEG-an-Open-Access-Arabic-Inner-Speech-EEG-Dataset-main/.gitignore
/kaggle/input/areeg-library/ArEEG-an-Open-Access-Arabic-Inner-Speech-EEG-Dataset-main/areeg-starter.ipynb
/kaggle/input/areeg-library/ArEEG-an-Open-Access-Arabic-Inner-Speech-EEG-Dataset-main/README.md
/kaggle/input/areeg-library/ArEEG-an-Open-Access-Arabic-Inner-Speech-EEG-Dataset-main/requirements.txt
/kaggle/input/areeg-library/ArEEG-an-Open-Access-Arabic-Inner-Speech-EEG-Dataset-main/dataloader.py
/kaggle/input/areeg-library/ArEEG-an-Open-Access-Arabic-Inner-Speech-EEG-Dataset-main/CITATION.cff
/kaggle/input/areeg-library/ArEEG-an-Open-Access-Arabic-Inner-Speech-EEG-Dataset-main/Utilities/Preprocessing.py
/kaggle/input/areeg-library/ArEEG-an-Open-Access-Arabic-Inner-Speech-EEG-Dataset-main/Utilities/__init__.py
/kaggle/input/areeg-library/ArEEG-an-Open-Access-Arabic-Inner-Speech-EEG-Dataset-mai

In [2]:
!ls /kaggle/input


arabic-eeg-sessions  areeg-library


In [9]:
!ls /kaggle/input/areeg-library


ArEEG-an-Open-Access-Arabic-Inner-Speech-EEG-Dataset-main


In [10]:
import os, sys

# Add ArEEG
sys.path.append('/kaggle/input/areeg-library/ArEEG-an-Open-Access-Arabic-Inner-Speech-EEG-Dataset-main/ArEEG')

# test extractor
from Utilities.Extractor import process_eeg
print("✅ ArEEG library imported successfully and ready!")


ModuleNotFoundError: No module named 'Utilities'

In [ ]:
import numpy as np
import os
from Utilities.Extractor import process_eeg

# ======================================
# STEP 1: Define dataset directory
# ======================================
# This folder contains all recorded EEG sessions for each subject
directory = "/kaggle/input/arabic-eeg-sessions/RecordedSessions"

# Number of samples per trial and the target EEG states (classes)
TIME_STEPS = 1200
included_states = ["Up", "Down", "Left", "Right", "Select"]

# Empty lists to collect all subjects' data
X_all, Y_all = [], []

# ======================================
# STEP 2: Loop through each subject folder
# ======================================
for folder in os.listdir(directory):
    subject_folder = os.path.join(directory, folder)
    if not os.path.isdir(subject_folder):
        continue

    print(f"📁 Processing {folder} ...")
    try:
        # Extract EEG trials and labels for this subject
        X, Y = process_eeg(
            TIME_STEPS=TIME_STEPS,
            included_states=included_states,
            subject_folder=subject_folder
        )

        print(f"   ✅ {len(Y)} valid trials | Labels: {set(Y)}")
        X_all.append(X)
        Y_all.append(Y)
    except Exception as e:
        print(f"   ⚠️ Skipped {folder} due to error: {e}")

# ======================================
# STEP 3: Combine all subjects into one dataset
# ======================================
X_all = np.concatenate(X_all, axis=0)
Y_all = np.concatenate(Y_all, axis=0)

print("\n✅ Files loaded successfully!")
print(f"📁 X_all shape: {X_all.shape}  (Trials, Channels, Time)")
print(f"📁 Y_all shape: {Y_all.shape}")
print("🎯 Unique labels:", np.unique(Y_all))

# ======================================
# STEP 4: Save arrays for later use
# ======================================
np.save("/kaggle/working/X_all.npy", X_all)
np.save("/kaggle/working/Y_all.npy", Y_all)


In [ ]:
# ======================================
# STEP 1: Save EEG data arrays
# ======================================
import numpy as np
import shutil

# Save both numpy arrays to the working directory
np.save("/kaggle/working/X_all.npy", X_all)
np.save("/kaggle/working/Y_all.npy", Y_all)

print("✅ EEG data saved as .npy files in /kaggle/working")

# ======================================
# STEP 2: Compress for download
# ======================================
# Zip both saved files into a single archive
shutil.make_archive("/kaggle/working/EEG_dataset", 'zip', "/kaggle/working")

print("📦 EEG_dataset.zip is ready for download!")


In [ ]:
# ======================================
# STEP 3: Visualize EEG signals
# ======================================
import numpy as np
import matplotlib.pyplot as plt

# Load saved data (optional if already loaded)
# X_all = np.load("/kaggle/working/X_all.npy", allow_pickle=True)
# Y_all = np.load("/kaggle/working/Y_all.npy", allow_pickle=True)

# Compute the mean signal across all trials (average EEG per channel)
mean_signal = X_all.mean(axis=0)  # shape (8, 1200)

# Define how many timesteps to display
ts = 100  # you can change it to 200, 500, etc.
timesteps = np.arange(ts)

# Create figure with 8 subplots (for 8 EEG channels)
fig, axes = plt.subplots(8, 1, figsize=(14, 16), sharex=True)

# Plot each EEG channel separately
for ch in range(mean_signal.shape[0]):
    axes[ch].plot(timesteps, mean_signal[ch, :ts])
    axes[ch].set_ylabel(f'Ch {ch+1}')
    axes[ch].grid(True)

# Add common labels and title
axes[-1].set_xlabel('Time Steps')
fig.suptitle('Average EEG Signals (first 100 time steps)', fontsize=14)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()


In [ ]:
# ======================================
# STEP 4: Split data into Train/Test sets
# ======================================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode text labels ('Up', 'Down', etc.) into numbers
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y_all)

# Split into 80% training and 20% testing
X_train, X_test, Y_train, Y_test = train_test_split(
    X_all, Y_encoded, test_size=0.2, random_state=42, stratify=Y_encoded
)

# Print shapes
print("✅ Data split successfully!")
print(f"Train shape: {X_train.shape},  Test shape: {X_test.shape}")
print(f"Train labels: {len(Y_train)},  Test labels: {len(Y_test)}")
print(f"Unique encoded labels: {label_encoder.classes_}")


In [ ]:
# ======================================
# STEP 5: EEGNet Model (Deep Learning)
# ======================================
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, SeparableConv2D
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# Reshape input for Conv2D: (samples, channels, time, 1)
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test_reshaped  = X_test.reshape(X_test.shape[0],  X_test.shape[1],  X_test.shape[2],  1)

# One-hot encode labels
Y_train_cat = to_categorical(Y_train)
Y_test_cat  = to_categorical(Y_test)

# EEGNet simplified architecture
model = Sequential([
    # First temporal convolution
    Conv2D(16, (1, 64), padding='same', activation='elu', input_shape=(8, 1200, 1)),
    BatchNormalization(),

    # Depthwise spatial convolution
    Conv2D(32, (8, 1), activation='elu', padding='valid'),
    BatchNormalization(),
    AveragePooling2D((1, 4)),
    Dropout(0.25),

    # Separable convolution
    SeparableConv2D(64, (1, 16), activation='elu', padding='same'),
    BatchNormalization(),
    AveragePooling2D((1, 8)),
    Dropout(0.25),

    # Classification layers
    Flatten(),
    Dense(64, activation='elu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  # 5 classes
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Summary
model.summary()


In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


In [ ]:
# ======================================
# STEP 6: Train EEGNet Model (CPU Version - Fixed)
# ======================================

# Train EEGNet model
history = model.fit(
    X_train_reshaped, Y_train_cat,
    validation_data=(X_test_reshaped, Y_test_cat),
    epochs=20,           # fewer epochs for CPU
    batch_size=16,       # smaller batch size for less memory usage
    verbose=1
)

# Save trained model
model.save("/kaggle/working/EEGNet_model_CPU.h5")

print("✅ Model trained and saved successfully (CPU Mode)!")


In [ ]:
# =========================================================
# 🧠 EEG Transformer Model — Final Fixed Version
# =========================================================

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# =========================================================
# STEP 1️⃣: Verify Labels
# =========================================================
print("✅ Labels already one-hot encoded!")
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

# =========================================================
# STEP 2️⃣: Prepare Input Data for Transformer
# =========================================================
X_train_tf = np.transpose(X_train, (0, 2, 1))  # (N, 1200, 8)
X_test_tf  = np.transpose(X_test, (0, 2, 1))   # (N, 1200, 8)
num_classes = Y_train.shape[1]

print("\n✅ Data ready for Transformer!")
print("Train shape:", X_train_tf.shape)
print("Test shape:", X_test_tf.shape)

# =========================================================
# STEP 3️⃣: Define Transformer Encoder
# =========================================================
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.3):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=head_size, dropout=dropout)(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Dense(ff_dim, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(inputs.shape[-1])(x)
    return x + res

# =========================================================
# STEP 4️⃣: Build Transformer Model
# =========================================================
input_shape = X_train_tf.shape[1:]
inputs = layers.Input(shape=input_shape)

x = transformer_encoder(inputs, head_size=64, num_heads=4, ff_dim=128)
x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=128)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model_tf = models.Model(inputs, outputs)
model_tf.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

print("\n✅ Transformer model summary:")
model_tf.summary()

# =========================================================
# STEP 5️⃣: Train the Model
# =========================================================
history_tf = model_tf.fit(
    X_train_tf, Y_train,
    validation_data=(X_test_tf, Y_test),
    epochs=30,
    batch_size=16,
    verbose=1
)

# =========================================================
# STEP 6️⃣: Evaluate the Model
# =========================================================
test_loss, test_acc = model_tf.evaluate(X_test_tf, Y_test, verbose=0)
print(f"\n🎯 Transformer Test Accuracy: {test_acc*100:.2f}%")
print(f"🧮 Transformer Test Loss: {test_loss:.4f}")

# =========================================================
# STEP 7️⃣: Save the Model
# =========================================================
model_tf.save("/kaggle/working/Transformer_model.h5")
print("\n💾 Model saved successfully → /kaggle/working/Transformer_model.h5")
